In [1]:
import pandas as pd
import numpy as np
import random 
np.random.seed = 42
random.seed(42)

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 100)
num_ofDesiredRecords = 100

In [2]:
def createRandomPhoneNumber_SAA():
    range_start = 10**9
    range_end = 10**10 -1
    num = str(random.randint(range_start,range_end))
    num_str = "{0} {1}-{2}".format(num[:3], num[3:7],num[7:])
    return num_str

In [3]:
def createRandomEmail(record,domainList=['@gmail.com','@stanfordalumni.org','@alumni.stanford.edu']):
    domain = random.choice(domainList)
    fName = record['first_name'].strip().lower()
    LName = record['last_name'].strip().lower()
    formats = []
    
    #Example: John Doe 123
    formats.append(fName[0]+LName+str(random.randint(10,9999))) #jdoe123
    formats.append(fName+LName+str(random.randint(10,9999))) #johndoe123
    formats.append(fName+LName[0]+str(random.randint(10,9999))) #johnd123
    formats.append(fName[0]+'.'+LName+str(random.randint(10,9999))) #j.doe123
    formats.append(fName+'.'+LName+str(random.randint(10,9999))) #john.doe123
    
    return random.choice(formats)+domain

In [4]:
def createDegreeString_SAA():
    degree_list = ['MS','MA','MBA','MD','PhD','BA','BS','JD','']
    degree_years = "'" + str(random.randint(80,99))
    degree = random.choice(degree_list) + ' ' + degree_years
    return degree.strip()

In [5]:
common_male_Fnames = [
    'Liam',
    'Noah',
    'Oliver',
    'Elijah',
    'James',
    'William',
    'Benjamin',
    'Lucas',
    'Henry',
    'Theodore',
]

In [6]:
common_female_Fnames = [
    'Mary',
    'Patricia',
    'Jennifer',
    'Linda',
    'Elizabeth',
    'Barbara',
    'Susan',
    'Jessica',
    'Sarah',
    'Karen',
]

In [7]:
city = ['Chicago', 'Boston',  'Madrid', 'Tokyo', 'Seoul', 'London','Beijing','Shanghai','Dubai','*',np.nan,'']
state = ['NY', 'WA', 'TX','CA','NM','*',np.nan,'']
country = ['Japan', 'United States', 'USA', 'China', 'UAE', 'Kuwait','*',np.nan,'']

In [8]:
common_FNames = []
common_FNames.extend(common_female_Fnames)
common_FNames.extend(common_male_Fnames)

In [9]:
common_LNames = ['Smith',
'Johnson',
'Williams',
'Brown',
'Jones',
'Garcia',
'Miller',
'Davis',
'Rodriguez',
'Martinez',]

In [10]:
df_SAA = pd.read_excel("./data/SAA Pride member reports headings.xlsx").append([np.nan]*num_ofDesiredRecords)

C:\Users\saads\AppData\Local\Temp\ipykernel_10524\2961325751.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SAA = pd.read_excel("./data/SAA Pride member reports headings.xlsx").append([np.nan]*num_ofDesiredRecords)


In [11]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(common_FNames))
    
df_SAA.first_name = __

In [12]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(common_LNames))
    
df_SAA.last_name = __

In [13]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choices([createRandomPhoneNumber_SAA(),'*',np.nan],weights=(1,5,5))[0])
    
df_SAA.home_phone_number = __

In [14]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(['*',np.nan,createRandomEmail(df_SAA.iloc[_])]))
    
df_SAA.home_email_address = __

In [15]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(['*',np.nan,createRandomEmail(df_SAA.iloc[_],['@stanfordalumni.org','@alumni.stanford.edu'])]))
    
df_SAA.saa_email_address = __

In [16]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choices([createRandomEmail(df_SAA.iloc[_]),np.nan],weights=(1, 50))[0])
    
df_SAA.gsb_email_address = __

In [17]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choices([createRandomEmail(df_SAA.iloc[_]),np.nan],weights=(1, 10))[0])
    
df_SAA.bus_email_address = __

In [18]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(city))
    
df_SAA.home_city = __

In [19]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(country))
    
df_SAA.home_country = __

In [20]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(country))
    
df_SAA.bus_country = __

In [21]:
state = {'Chicago': 'IL', 
           'Boston': 'MA', 
           'New York' : 'NY', 
           'San Francisco': 'CA', 
           'Los Angeles' : 'CA', 
           'Austin' : 'TX',
        'Dallas': 'TX',
        'Denver': 'CO',
        '':'',
        '*':'*'}

df_SAA.home_state_code = df_SAA['home_city'].map(state)

In [22]:
__  = []
for _ in range(0,num_ofDesiredRecords):
    __.append(random.choice(['',np.nan,random.randint(1990, 2018)]))
    
df_SAA.pref_class_year = __


In [23]:
degrees = []
for i in range(0,num_ofDesiredRecords):
    num_draw = random.randint(0,3)
    degree = np.NaN
    k=0
    while k < num_draw:
        if k == 0:
            degree = createDegreeString_SAA()
        else:
            degree = degree +', '+ createDegreeString_SAA()
        k+=1
    degrees.append(degree)
    
df_SAA.short_degree_string = degrees

In [27]:
df_SAA.to_excel('data/SAA_FakeDB.xlsx',index = False)